# Panda's Cookbook

This file has the goal to provide all the cookbook codes along with commentary and explanations about what's happening in every code, when needed. This is not supposed to be an introduction to pandas, just a file to look up to when in doubt about a function and its use.

In [2]:
import pandas as pd
import numpy as np

### Idioms

- **.loc()** -> Access a group of rows and columns by label(s) or a boolean array.
- **.iloc()** -> Access group of rows and columns by integer position(s).
- **.mask()** -> Replace values where the condition is True.

- **.where()** -> Replace values where the condition is False.

In [24]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [25]:
# If-then assignment on one column:
df.loc[df.AAA >= 7, "BBB"] = -1
df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,-1,-50


In [26]:
# If-then assignment to more than one column:
df.loc[df.AAA >= 5, ["BBB", "CCC"]] = 555
df

,AAA,BBB,CCC
0,4,10,100
1,5,555,555
2,6,555,555
3,7,555,555


In [35]:
df_mask = pd.DataFrame(
    {"AAA": [True] * 4, "BBB": [False] * 4, "CCC": [True, False] * 2}
)
print(df_mask)
df.where(df_mask, -1000)

    AAA    BBB    CCC
0  True  False   True
1  True  False  False
2  True  False   True
3  True  False  False


,AAA,BBB,CCC
0,4,-1000,100
1,5,-1000,-1000
2,6,-1000,555
3,7,-1000,-1000


In [42]:
# if-then-else using NumPy's where()
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df["logic"] = np.where(df["AAA"] > 5, "high", "low")
df

,AAA,BBB,CCC,logic
0,4,10,100,low
1,5,20,50,low
2,6,30,-30,high
3,7,40,-50,high


### Splitting

In [43]:
df[df['AAA'] <= 5]

,AAA,BBB,CCC,logic
0,4,10,100,low
1,5,20,50,low


In [44]:
df[df.AAA > 5]

,AAA,BBB,CCC,logic
2,6,30,-30,high
3,7,40,-50,high


### Building Criteria

In [45]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)

df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [53]:
# and (without assignment returns a Series)
print(df.loc[(df["BBB"] < 25) & (df["CCC"] >= -40), "AAA"], "\n")

# or (without assignment returns a Series)
print(df.loc[(df["BBB"] > 25) | (df["CCC"] >= -40), "AAA"])

0    4
1    5
Name: AAA, dtype: int64 

0    4
1    5
2    6
3    7
Name: AAA, dtype: int64


In [54]:
# or (with assignment modifies the DataFrame.)
df.loc[(df["BBB"] > 25) | (df["CCC"] >= 75), "AAA"] = 0.1

df

,AAA,BBB,CCC
0,0.1,10,100
1,5.0,20,50
2,0.1,30,-30
3,0.1,40,-50


In [65]:
# Select rows with data closest to certain value using argsort
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
aValue = 1
df.loc[(df.CCC - aValue).abs().argsort()]

,AAA,BBB,CCC
2,6,30,-30
1,5,20,50
3,7,40,-50
0,4,10,100


In [69]:
# Dynamically reduce a list of criteria using a binary operators
df = pd.DataFrame(

    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}

)

Crit1 = df.AAA <= 5.5

Crit2 = df.BBB == 10.0

Crit3 = df.CCC > -40.0

# You can either hardcode:
AllCrit = Crit1 & Crit2 & Crit3

# or:

import functools

CritList = [Crit1, Crit2, Crit3]

AllCrit = functools.reduce(lambda x, y: x & y, CritList)

df[AllCrit]

,AAA,BBB,CCC
0,4,10,100


### Selection

In [75]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)

# Using both row labels and value conditionals
df[(df.AAA <= 6) & (df.index.isin([0, 2, 4]))]

,AAA,BBB,CCC
0,4,10,100
2,6,30,-30


Differences between .loc() and .iloc()

In [80]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]},
    index=["foo", "bar", "boo", "kar"],
)

# Label
df.loc["bar":"kar"]

,AAA,BBB,CCC
bar,5,20,50
boo,6,30,-30
kar,7,40,-50


In [79]:
# Generic
df[0:3]

df["bar":"kar"]

,AAA,BBB,CCC
bar,5,20,50
boo,6,30,-30
kar,7,40,-50


In [81]:
# Ambiguity arises when an index consists of integers with a non-zero start or non-unit increment.
data = {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
df2 = pd.DataFrame(data=data, index=[1, 2, 3, 4]) # Note index starts at 1.

df2.iloc[1:3]  # Position-oriented

,AAA,BBB,CCC
2,5,20,50
3,6,30,-30


In [82]:
df2.loc[1:3]  # Label-oriented

,AAA,BBB,CCC
1,4,10,100
2,5,20,50
3,6,30,-30


In [83]:
# Using inverse operator (~) to take the complement of a mask
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)

df

,AAA,BBB,CCC
0,4,10,100
1,5,20,50
2,6,30,-30
3,7,40,-50


In [84]:
df[~((df.AAA <= 6) & (df.index.isin([0, 2, 4])))]

,AAA,BBB,CCC
1,5,20,50
3,7,40,-50


### New Columns

In [91]:
df = pd.DataFrame({"AAA": [1, 2, 1, 3], "BBB": [1, 1, 2, 2], "CCC": [2, 1, 3, 1]})

source_cols = df.columns  # Or some subset would work too

new_cols = [str(x) + "_cat" for x in source_cols]

categories = {1: "Alpha", 2: "Beta", 3: "Charlie"}

df[new_cols] = df[source_cols].applymap(categories.get)

df

,AAA,BBB,CCC,AAA_cat,BBB_cat,CCC_cat
0,1,1,2,Alpha,Alpha,Beta
1,2,1,1,Beta,Alpha,Alpha
2,1,2,3,Alpha,Beta,Charlie
3,3,2,1,Charlie,Beta,Alpha


In [105]:
# Keep other columns when using min() with groupby
# Method 1 : idxmin() to get the index of the minimums
df = pd.DataFrame(
    {"AAA": [1, 1, 1, 2, 2, 2, 3, 3], "BBB": [2, 1, 3, 4, 5, 1, 2, 3]}
)

df.loc[df.groupby("AAA")["BBB"].idxmin()]

,AAA,BBB
1,1,1
5,2,1
6,3,2


In [110]:
# Method 2 : sort then take first of each
df.sort_values(by="BBB").groupby("AAA", as_index=False).first()

,AAA,BBB
0,1,1
1,2,1
2,3,2


In [125]:
df.sort_values("BBB").groupby("AAA", as_index=False).first()

,AAA,BBB
0,1,1
1,2,1
2,3,2


In [130]:
df.sort_values("AAA").groupby("BBB", as_index=False).first()

,BBB,AAA
0,1,1
1,2,1
2,3,1
3,4,2
4,5,2
